<a href="https://colab.research.google.com/github/clara-lan/Capstone/blob/main/CapstoneSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use SVM to detect empty space and pre-process images

# Step1: Import system and dependancies

In [1]:
import os
from os import listdir
from os.path import isfile, join
!pip install roboflow
from roboflow import Roboflow
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67 kB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 34.4 MB/s 
     |████████████████████████████████| 145 kB 56.7 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 138 kB 10.9 MB/s 
     |████████████████████████████████| 178 kB 45.6 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=56995679db77c7c3bf13483cadccf18b7d91af1c2db72636f2bdb0ec30eb0f32
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: cer

In [3]:
import pandas as pd
import cv2
import os
import sys
import math
import time
from sklearn import svm
from sklearn.metrics import accuracy_score
from skimage.feature import hog

## Import dataset from Roboflow

In [4]:
from sklearn.utils.fixes import sklearn
import tensorflow.python.platform
from tensorflow.python.platform import gfile
"""Read method1: Load roboflow dataset from Roboflow in tfrecord"""

rf = Roboflow(api_key="cKIuGvQRsLbBvFgxNztc")
project = rf.workspace("myworkspace-nfnwm").project("capstone-nyjby")
dataset = project.version(3).download("tfrecord")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Capstone-3 in tfrecord:: 100%|██████████| 11/11 [00:00<00:00, 23.09it/s]


In [5]:
def extract_images(path,mode):
  """
    Function: extract_images
    Input: path - valid image path in tf record
           mode - 1 for training, 2 for test
    Output: image dict, {key: {image, label}}, key is index
    Description: This function extract images and labels from tfrecord, maps them to corresponding index
  """
  images = {}
  # Create a description of the features.
  features = {
      "image/encoded": tf.io.FixedLenFeature([], tf.string),
      "image/format": tf.io.FixedLenFeature([], tf.string),
      "image/height": tf.io.FixedLenFeature([], tf.int64),
      "image/object/bbox/xmax": tf.io.VarLenFeature(tf.float32),
      "image/object/bbox/xmin": tf.io.VarLenFeature(tf.float32),
      "image/object/bbox/ymax": tf.io.VarLenFeature(tf.float32),
      "image/object/bbox/ymin": tf.io.VarLenFeature(tf.float32),
      "image/object/class/label": tf.io.VarLenFeature(tf.int64),
      "image/object/class/text": tf.io.VarLenFeature(tf.string),
      "image/width": tf.io.FixedLenFeature([], tf.int64),
  }
  size = 0
  idx = 0
  if mode == 1:
    size = 400
  elif mode == 2:
    size = 100
  train_dataset = tf.data.TFRecordDataset(path)
  for raw_record in train_dataset.take(size):
    sample = tf.io.parse_single_example(raw_record, features)
    image = tf.image.decode_image(sample['image/encoded'], dtype=tf.float32) 
    label = sample['image/object/class/label']
    images[idx] = [image, label]
    idx+=1
  return images
# path =  '/content/Capstone-3/train/empty-shelf-space.tfrecord'
# extract_images(path, 1)

# Step2: Preprocess data

In [6]:
# test_dataset = tf.data.TFRecordDataset('/content/Empty-shelf-detection-15/test/Empty-space.tfrecord')
# valid_dataset = tf.data.TFRecordDataset('/content/Empty-shelf-detection-15/valid/Empty-space.tfrecord')
train_labels = '/content/Capstone-3/train/empty-shelf-space_label_map.pbtxt'
# valid_labels = '/content/Empty-shelf-detection-15/valid/Empty-space_label_map.pbtxt'

# Step3: Train SVM
---



In [7]:
# rnoMapping = {}

def datagen(mode):
    """
    Function: datagen 
    
    Input: 
        mode - 1 denotes train data ; 2 denotes test data
    
    Output: Train/Test data and labels depending on mode value
    
    Description: This function computes HOG features for each image in the Dataset/train or Dataset/test folder, assigns label to the descriptor vector of the image and returns the final train/test data and labels matrices used for feeding the SVM in training phase or predicting the label of test data.
    
    """

    X = []
    y = []
    # TODO: remove the dummy index to select y
    idx = 0
    cnt = 0
    if mode == 1:
        path = '/content/Capstone-3/train/empty-shelf-space.tfrecord'
    elif mode == 2:
        path = '/content/Capstone-3/valid/empty-shelf-space.tfrecord'
    # read image
    images = extract_images(path, mode)
    for key in images.keys():
        # compute HOG features
        img, label = images[key][0], images[key][1]
        # print(type(img))
        des, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(4, 4), block_norm= 'L2',visualize=True)
        # print("des: ", des)

        """Read labels"""
        # if mode == 1:
        #     # construct dictionary for roll no. mapping
        #   if label not in rnoMapping.keys():
        #       rnoMapping[label] = cnt
        #       cnt += 1

        # append descriptor and label to train/test data, labels
        X.append(des)
        label = tf.sparse.to_dense(label).numpy().tolist()
        y.append(label)
    # print("X: ",X)
        # print("y: ", label)
    # return data and label
    return X, y
# datagen(1)
# datagen(2)


In [9]:
def train_svm():
    # list of training and test files
    # call 'datagen' function to get training and testing data & labels
    mlb = MultiLabelBinarizer()
    Xtrain, ytrain = datagen(1)
    Xtest, ytest = datagen(2)
    # print("ytrain: ", ytrain)
    # convert matrices to numpy array for fast computation
    Xtrain = np.array(Xtrain)
    # convert labels to 1d array to enabel sklearn
    ytrain = [item[0] for item in ytrain]
    # print("x: ", len(Xtrain))
    # print("ytrain: ", ytrain)
    Xtest = np.array(Xtest)
    ytest = [item[0] for item in ytest]

    # training phase: SVM , fit model to training data ------------------------------
    clf = svm.SVC(kernel = 'linear')
    clf.fit(Xtrain, ytrain)
    # predict labels for test data
    ypred = clf.predict(Xtest)
    
    # compute accuracy
    accuracy = accuracy_score(ytest, ypred) * 100
    print("\nAccuracy: %.2f" % accuracy + "%")
start_time = time.time()
train_svm()
print('Execution time: %.2f' % (time.time() - start_time) + ' seconds\n')


Accuracy: 82.00%
Execution time: 84.81 seconds

